In [7]:
!pip install --upgrade gspread oauth2client

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from google.colab import files
uploaded = files.upload()

Saving automated-email-referral-b4f652a49a21.json to automated-email-referral-b4f652a49a21 (1).json


In [ ]:
def get_sheet_data(json_keyfile='sample.json', sheet_name='Recruiters'):
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive"
    ]

    creds = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
    client = gspread.authorize(creds)
    sheet = client.open(sheet_name).sheet1

    data = sheet.get_all_records()
    df = pd.DataFrame(data)
    return df


In [10]:
df = get_sheet_data()
df.head()

,Name,Email,Job ID/Link,Company
0,Yuvraj Agrawal,yuvraja@adobe.com,,Adobe
1,Shivam Jaiswal,sjaiswal@adobe.com,,Adobe
2,Sohan Anisetty,sanisetty@adobe.com,,Adobe
3,Pranav Sukhijia,sukhija@adobe.com,,Adobe
4,Saumya Shrivastava,sshrivastava@adobe.com,,Adobe


In [11]:
TRACK_FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/last_row.txt"

os.makedirs(os.path.dirname(TRACK_FILE_PATH), exist_ok=True)

In [15]:
def get_last_row_index():
    if not os.path.exists(TRACK_FILE_PATH):
        return 0
    with open(TRACK_FILE_PATH, 'r') as f:
        content = f.read().strip()
        if not content:  # Check if the file is empty
            return 0
        return int(content)

def update_last_row_index(new_index):
    with open(TRACK_FILE_PATH, 'w') as f:
        f.write(str(new_index))

def get_new_rows(df):
    last_index = get_last_row_index()
    new_index = len(df)

    if new_index > last_index:
        new_rows = df.iloc[last_index:new_index]
        update_last_row_index(new_index)
        return new_rows
    return pd.DataFrame()

In [16]:
df = get_sheet_data()
new_df = get_new_rows(df)

if not new_df.empty:
    print("New entries found:")
    print(new_df)
else:
    print("No new rows to process.")

New entries found:
                   Name                     Email Job ID/Link Company
0        Yuvraj Agrawal         yuvraja@adobe.com               Adobe
1        Shivam Jaiswal        sjaiswal@adobe.com               Adobe
2        Sohan Anisetty       sanisetty@adobe.com               Adobe
3       Pranav Sukhijia         sukhija@adobe.com               Adobe
4    Saumya Shrivastava    sshrivastava@adobe.com               Adobe
..                  ...                       ...         ...     ...
100         Yash Parmar          yparmar@uber.com                Uber
101     Vishal Kaloriya  vishal.kaloriya@uber.com                Uber
102     Saritesh Tiwari         saritesh@uber.com                Uber
103       Ishan Singhal    ishan.singhal@uber.com                Uber
104            Yash Raj         yash.raj@uber.com                Uber

[105 rows x 4 columns]


In [17]:
import smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [ ]:
# Email credentials
SENDER_EMAIL = "rujuldwivedi@icloud.com"
APP_PASSWORD = "sample_password" 

# Resume file
RESUME_PATH = "/content/drive/MyDrive/Documents/Rujul_Dwivedi_Resume.pdf"

# Portfolio and links
PORTFOLIO_LINK = "https://rujuldwivedi.in"
LINKEDIN_LINK = "https://linkedin.com/in/rujuldwivedi"
GITHUB_LINK = "https://github.com/rujuldwivedi"

In [19]:
def send_referral_email(to_email, name, company, job_id=None, job_link=None):
    msg = MIMEMultipart()
    subject = f"Application for SWE role at {company}"
    if job_id and not pd.isna(job_id):
        subject += f" – Job ID: {job_id}"
    msg['Subject'] = subject
    msg['From'] = SENDER_EMAIL
    msg['To'] = to_email

    # Compose body
    body = f"""
Hi {name}, I hope this email finds you well.

I am writing to express my keen interest in the Software Engineering position{f" {job_link}" if job_link and not pd.isna(job_link) else ""} at {company}. As a recent graduate with a strong foundation in mathematics and computer science, I am eager to contribute to your team's success.

My experience includes developing and deploying a BERT + LSTM NLP pipeline on enterprise-scale imbalanced data for Databricks during my recent internship and applied machine learning for option pricing using PINNs and LSTMs.
On the software engineering front, I have led the development of full-stack platforms using Spring Boot, MERN, and Django, and built production-ready systems with technologies like Kafka, Redis, Docker, and Firebase. I am also thorough with Core Programming and Data Structures and Algorithms.

I would appreciate the opportunity to discuss how my skills and experience can add value to {company}. Please feel free to reach out if you require any additional information.

P.S. I’ve linked my resume for your reference. Thank you for your time and consideration.

Warm regards,
Rujul Dwivedi

📞 +91 96951 33900
📧 rujuldwivedi@icloud.com
🔗 [Portfolio]({PORTFOLIO_LINK}) | [LinkedIn]({LINKEDIN_LINK}) | [GitHub]({GITHUB_LINK})
"""

    # Attach body as HTML/plain text
    msg.attach(MIMEText(body, "plain"))

    # Attach resume
    with open(RESUME_PATH, "rb") as f:
        part = MIMEApplication(f.read(), Name="Rujul_Resume.pdf")
    part['Content-Disposition'] = 'attachment; filename="Rujul_Resume.pdf"'
    msg.attach(part)

    # Send the email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(SENDER_EMAIL, APP_PASSWORD)
        server.send_message(msg)

    print(f"Email sent to {name} ({to_email}) at {company}")


In [ ]:
# Use iCloud SMTP settings
SMTP_SERVER = "smtp.mail.me.com"
SMTP_PORT = 587

EMAIL_ADDRESS = "rujuldwivedi@icloud.com"
APP_PASSWORD = "sample_password"

In [21]:
def send_referral_email(to_email, name, company, job_id=None, job_link=None):
    msg = MIMEMultipart()
    subject = f"Application for SWE role at {company}"
    if job_id and not pd.isna(job_id):
        subject += f" – Job ID: {job_id}"
    msg['Subject'] = subject
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = to_email

    body = f"""
Hi {name}, I hope this email finds you well.

I am writing to express my keen interest in the Software Engineering position{f" {job_link}" if job_link and not pd.isna(job_link) else ""} at {company}. As a recent graduate with a strong foundation in mathematics and computer science, I am eager to contribute to your team's success.

My experience includes developing and deploying a BERT + LSTM NLP pipeline on enterprise-scale imbalanced data for Databricks during my recent internship and applied machine learning for option pricing using PINNs and LSTMs.
On the software engineering front, I have led the development of full-stack platforms using Spring Boot, MERN, and Django, and built production-ready systems with technologies like Kafka, Redis, Docker, and Firebase. I am also thorough with Core Programming and Data Structures and Algorithms.

I would appreciate the opportunity to discuss how my skills and experience can add value to {company}. Please feel free to reach out if you require any additional information.

P.S. I’ve linked my resume for your reference. Thank you for your time and consideration.

Warm regards,
Rujul Dwivedi

📞 +91 96951 33900
📧 rujuldwivedi@icloud.com
🔗 [Portfolio]({PORTFOLIO_LINK}) | [LinkedIn]({LINKEDIN_LINK}) | [GitHub]({GITHUB_LINK})
"""
    msg.attach(MIMEText(body, "plain"))

    # Attach resume
    with open(RESUME_PATH, "rb") as f:
        part = MIMEApplication(f.read(), Name="Rujul_Resume.pdf")
    part['Content-Disposition'] = 'attachment; filename="Rujul_Resume.pdf"'
    msg.attach(part)

    # Send using iCloud SMTP
    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_ADDRESS, APP_PASSWORD)
        server.send_message(msg)

    print(f"Email sent to {name} ({to_email}) at {company}")


In [22]:
import time

def process_and_send_all_new_emails():
    print("Checking for new entries...")
    df = get_sheet_data()
    new_rows = get_new_rows(df)

    if new_rows.empty:
        print("No new rows found.")
        return

    for _, row in new_rows.iterrows():
        try:
            send_referral_email(
                to_email=row['Email'],
                name=row['Name'],
                company=row['Company'],
                job_id=row.get('Job ID'),
                job_link=row.get('Job Link')
            )
        except Exception as e:
            print(f"Failed to send to {row['Email']}: {e}")

# Manual trigger (for now)
process_and_send_all_new_emails()


Checking for new entries...
No new rows found.


In [24]:
!pip install schedule

In [26]:
import time

while True:
    process_and_send_all_new_emails()
    print("⏳ Sleeping for 1 hour...\n")
    time.sleep(3600)

Checking for new entries...
No new rows found.
⏳ Sleeping for 1 hour...

Checking for new entries...
No new rows found.
⏳ Sleeping for 1 hour...



KeyboardInterrupt: 